# Evrişimli sinir ağları

Sinir ağlarının imgelerle başa çıkmada oldukça iyi olduğunu ve tek katmanlı algılayıcının bile MNIST veri kümesinden el yazısı rakamları makul bir doğrulukla tanıyabildiğini daha önce gördük. Ancak, MNIST veri kümesi çok özeldir ve tüm rakamlar imgenin içinde ortalanmıştır, bu da görevi kolaylaştırır.

Gerçek hayatta, resimdeki nesneleri, resimdeki tam konumlarından bağımsız olarak tanıyabilmek istiyoruz. Bilgisayarla görme, genel sınıflandırmadan farklıdır çünkü resimde belirli bir nesneyi bulmaya çalışırken, belirli **örüntüleri** ve bunların kombinasyonlarını arayarak imgeyi tarıyoruz. Örneğin, bir kedi ararken, önce bıyık oluşturabilen yatay çizgilere bakabiliriz ve ardından belirli bir bıyık kombinasyonu bize bunun aslında bir kedi resmi olduğunu söyleyebilir. İmge üzerindeki tam konumları değil, belirli örüntülerin göreli konumu ve varlığı önemlidir.

Örüntüleri çıkarmak için **evrişimli filtreler** kavramını kullanacağız. Ama önce önceki ünitelerde tanımladığımız tüm bağımlılıkları ve fonksiyonları yükleyelim. Kodu kısa ve temiz tutmak için bu not defterinde tanımlamak istemediğimiz bazı yararlı işlevleri içeren tfcv yardımcı kütüphanesini de içe aktaracağız.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from tfcv import *

Bu örnekte daha önce gördüğümüz MNIST veri kümesine ve imge sınıflandırmasına odaklanacağız. Keras yerleşik fonksiyonlarını kullanıp veri kümesini yükleyerek başlayacağız.

In [ ]:
(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

## Evrişimli filtreler

Evrişimli filtreler, imgenin her pikseli üzerinde çalışan ve komşu piksellerin ağırlıklı ortalamasını hesaplayan küçük pencerelerdir.


Ağırlık katsayılarının matrisleriyle tanımlanırlar. MNIST el yazısı rakamlarımız üzerinde iki farklı evrişimli filtre uygulama örneklerini görelim:

In [ ]:
plot_convolution(x_train[:5],[[-1.,0.,1.],[-1.,0.,1.],[-1.,0.,1.]],'Dikey kenar filtresi')
plot_convolution(x_train[:5],[[-1.,-1.,-1.],[0.,0.,0.],[1.,1.,1.]],'Yatay kenar filtresi')

İlk filtre **dikey kenar filtresi** olarak adlandırılır ve aşağıdaki matris ile tanımlanır:
$$
\left(
    \begin{matrix}
     -1 & 0 & 1 \cr
     -1 & 0 & 1 \cr
     -1 & 0 & 1 \cr
    \end{matrix}
\right)
$$
Bu filtre nispeten düzgün bir piksel alanı üzerinden geçtiğinde, tüm değerler 0'a toplar. Ancak, imgede dikey bir kenarla karşılaştığında, yüksek uç değer üretilir. Bu nedenle yukarıdaki imgelerde dikey kenarların yüksek ve düşük değerlerle temsil edildiğini, yatay kenarların ise ortalamasının alındığını görebilirsiniz.

Yatay kenar filtresi uyguladığımızda tam tersi bir şey olur - yatay çizgiler büyütülür ve dikey çizgiler ortalanır.

Klasik bilgisayarla görmede, öznitelikler oluşturmak için imgeye birden çok filtre uygulandı ve bunlar daha sonra bir sınıflandırıcı oluşturmak için makine öğrenmesi algoritması tarafından kullanıldı. Bu filtreler aslında bazı hayvanların görme sistemlerinde bulunan sinir yapılarına benzer.

<img src="../images/lmfilters.jpg" width="400"/>

Bununla birlikte, derin öğrenmede, sınıflandırma problemini çözmek için en iyi evrişimli filtreleri **öğrenen** ağlar oluştururuz. Bunu yapmak için **evrişimli katmanları** tanıtıyoruz.

## Evrişimli Katmanlar

Evrişimli katmanın ağırlıklarını eğitilebilir hale getirmek için, imgeye evrişimli filtre penceresi uygulama sürecini bir şekilde matris işlemlerine indirgememiz gerekir, bu işlemler daha sonra geri yayma eğitimine tabi olabilir. Bunu yapmak için **im2col** adını verdiğimiz akıllı bir matris dönüşümü kullanıyoruz.

Aşağıdaki piksellere sahip küçük bir $\mathbf{x}$ imgemiz olduğunu varsayalım:

$$
\mathbf{x} = \left(
         \begin{array}{ccccc}
           a & b & c & d & e \\
           f & g & h & i & j \\
           k & l & m & n & o \\
           p & q & r & s & t \\
           u & v & w & x & y \\
         \end{array}
     \right)
$$

Ayrıca aşağıdaki ağırlıklara sahip iki dönüştürme filtresini uygulamak istiyoruz:
$$
W^{(i)} = \left(\begin{array}{ccc}
            w^{(i)}_{00} & w^{(i)}_{01} & w^{(i)}_{02} \\
            w^{(i)}_{10} & w^{(i)}_{11} & w^{(i)}_{12} \\
            w^{(i)}_{20} & w^{(i)}_{21} & w^{(i)}_{22} \\
            \end{array}\right) 
$$

Evrişimi uygularken, sonucun ilk pikseli, öğe bazında çarpılarak elde edilir.
$\left(\begin{array}{ccc}
  a & b & c \\
  f & g & h \\
  k & l & m \\
\end{array}\right)$  $W^{(i)}$, ikinci eleman bununla çarparak $\left(\begin{array}{ccc}
  b & c & d \\
  g & h & i \\
  l & m & n \\
\end{array}\right)$ $W^{(i)}$, ve böyle devam eder.

Bu işlemi formüle dökmek için, $x$ orijinal imgenin tüm $3\times3$ parçalarını aşağıdaki matrise çıkaralım:

$$
\mathrm{im2col}(x) = \left[
        \begin{array}{cccccc}
          a & b & \ldots & g & \ldots & m \\
          b & c & \ldots & h & \ldots & n \\
          c & d & \ldots & i & \ldots & o \\
          f & g & \ldots & l & \ldots & r \\
          g & h & \ldots & m & \ldots & s \\
          h & i & \ldots & n & \ldots & t \\
          k & l & \ldots & q & \ldots & w \\
          l & m & \ldots & r & \ldots & x \\
          m & n & \ldots & s & \ldots & y \\
        \end{array}
    \right]
$$

Bu matrisin her bir sütunu, orijinal imgenin her bir $3\times3$ alt bölgesine karşılık gelir. Şimdi, evrişimin sonucunu elde etmek için, bu matrisi matris veya ağırlıklarla çarpmamız yeterlidir.

$$
\mathbf{W} = \left[
         \begin{array}{cccccccc}
            w^{(0)}_{00} & w^{(0)}_{01} & w^{(0)}_{02} & w^{(0)}_{10} & w^{(0)}_{11} & \ldots & w^{(0)}_{21} & w^{(0)}_{22} \\
            w^{(1)}_{00} & w^{(1)}_{01} & w^{(1)}_{02} & w^{(1)}_{10} & w^{(1)}_{11} & \ldots & w^{(1)}_{21} & w^{(1)}_{22} \\
         \end{array}
       \right]
$$
(bu matrisin her satırı, bir satıra düzleştirilmiş $i$. filtrenin ağırlıklarını içerir)

Bu nedenle, orijinal imgeye bir evrişim filtresinin uygulanması, geri yayma kullanarak nasıl başa çıkacağımızı zaten bildiğimiz matris çarpımı ile değiştirilebilir:
$$
C(x) = W\times\mathbf{im2col}(x)
$$

Evrişimli katmanlar, `Conv2d` sınıfı kullanılarak tanımlanır. Aşağıdakileri belirtmemiz gerekiyor:
* `filters` - kullanılacak filtre sayısı. 9 farklı filtre kullanacağız, bu da ağa senaryomuz için hangi filtrelerin en iyi şekilde çalıştığını keşfetme fırsatı verecektir.
* `kernel_size` kayan pencerenin boyutudur. Genellikle 3x3 veya 5x5 filtreler kullanılır.

En basit CNN, bir evrişimli katman içerecektir. 28x28 girdi boyutu verildiğinde, dokuz adet 5x5 filtre uyguladıktan sonra 24x24x9'luk bir tensör elde edeceğiz. Uzamsal boyut daha küçüktür, çünkü 5 uzunluğundaki bir kayma aralığının 28 piksele sığabileceği yalnızca 24 konum vardır.

Evrişimden sonra, 24x24x9 tensörü 5184 boyutunda tek bir vektöre düzleştiriyoruz ve ardından 10 sınıf üretmek için doğrusal katman ekliyoruz. Ayrıca katmanlar arasında `relu` etkinleştirme fonksiyonunu kullanıyoruz.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=9, kernel_size=(5,5), input_shape=(28,28,1),activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['acc'])

model.summary()

Tam bağlı çok katmanlı ağlardaki yaklaşık 80 bin parametre ile karşılaştırıldığında, bu ağın yaklaşık 50 bin eğitilebilir parametre içerdiğini görebilirsiniz. Bu, daha küçük veri kümelerinde bile iyi sonuçlar elde etmemizi sağlar çünkü evrişimli ağlar çok daha iyi genelleşir.

> **Not**: Pratik durumların çoğunda, renkli imgelere evrişimli katmanlar uygulamak isteriz. Bu nedenle, `Conv2D` katmanı girdinin $W\times H\times C$ şeklinde olmasını bekler; burada $W$ ve $H$ imgenin genişliği ve yüksekliğidir ve $C$ renk kanallarının sayısıdır . Gri tonlamalı imgeler için $C=1$ ile aynı şekle ihtiyacımız var.

Eğitime başlamadan önce verilerimizi yeniden şekillendirmemiz gerekiyor:

In [ ]:
x_train_c = np.expand_dims(x_train,3)
x_test_c = np.expand_dims(x_test,3)
hist = model.fit(x_train_c,y_train,validation_data=(x_test_c,y_test),epochs=5)

In [ ]:
plot_results(hist)

Gördüğünüz gibi, önceki üniteden tam bağlı ağlara kıyasla daha yüksek doğruluk ve çok daha fazla hız (dönem sayısı açısından) elde edebiliyoruz. Ancak, eğitimin kendisi daha fazla kaynak gerektirir ve GPU olmayan bilgisayarlarda daha yavaş olabilir.

## Evrişimli Katmanları Görselleştirme

Neler olup bittiğini biraz daha anlamaya çalışmak için eğitilmiş evrişimli katmanlarımızın ağırlıklarını da görselleştirebiliriz:

In [ ]:
fig,ax = plt.subplots(1,9)
l = model.layers[0].weights[0]
for i in range(9):
    ax[i].imshow(l[...,0,i])
    ax[i].axis('off')

Bu filtrelerden bazılarının bazı eğik hareketleri tanıyor gibi göründüğünü, diğerlerinin ise oldukça rastgele göründüğünü görebilirsiniz.

> **Görev**: Aynı ağı 3x3 filtrelerle eğitin ve görselleştirin. Daha tanıdık örüntüler görüyor musunuz?

## Çok katmanlı CNN'ler ve ortaklama katmanları

İlk evrişimli katmanlar, yatay veya dikey çizgiler gibi ilkel örüntüleri arar, ancak ilkel şekiller gibi daha yüksek seviyeli örüntüleri aramak için bunların üzerine başka evrişimli katmanlar uygulayabiliriz. Daha sonra daha fazla evrişimli katman, bu şekilleri, sınıflandırmaya çalıştığımız son nesneye kadar resmin bazı kısımlarında birleştirebilir.

Bunu yaparken bir numara da uygulayabiliriz: İmgenin uzamsal boyutunu azaltmak. Kayan 3x3 pencerede yatay bir hareket olduğunu tespit ettikten sonra, bunun tam olarak hangi pikselde meydana geldiği çok önemli değildir. Böylece **ortaklama katmanlarından** birini kullanarak imgenin boyutunu "küçültebiliriz":

* **Ortalama Ortaklama** kayan bir pencere alır (örneğin, 2x2 piksel) ve pencere içindeki değerlerin ortalamasını hesaplar.
* **Azami (Maksimum) Ortaklama**, pencereyi maksimum değerle değiştirir. Maksimum ortaklamanın arkasındaki fikir, kayan pencerede belirli bir örüntünün varlığını tespit etmektir.

Bu nedenle, tipik bir CNN'de, imgenin boyutlarını azaltmak için aralarında ortaklama katmanları olan birkaç evrişimli katman olacaktır. Ayrıca filtre sayısını da artıracağız, çünkü örüntüler daha gelişmiş hale geldikçe aramamız gereken daha olası ilginç kombinasyonlar vardır.

![Ortaklama katmanlarına sahip birkaç evrişimli katmanı gösteren bir imge.](../images/cnn-pyramid.png)

Uzamsal boyutların küçülmesi ve öznitelik/filtre boyutlarının artması nedeniyle bu mimariye **piramit mimarisi** de denir.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=10, kernel_size=(5,5), input_shape=(28,28,1),activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(filters=20, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D(),    
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['acc'])

model.summary()

Eğitilebilir parametre sayısının (~8.5 bin) önceki durumlardan çok daha az olduğuna dikkat edin. Bunun nedeni, genel olarak evrişimli katmanların birkaç parametreye sahip olması ve son yoğun katman uygulanmadan önce imgenin boyutsallığının önemli ölçüde azalmasıdır. Az sayıda parametrenin modellerimiz üzerinde olumlu etkisi vardır çünkü bu, daha küçük veri kümesi boyutlarında bile aşırı öğrenmeyi önlemeye yardımcı olur.

In [ ]:
hist = model.fit(x_train_c,y_train,validation_data=(x_test_c,y_test),epochs=5)

In [ ]:
plot_results(hist)

Muhtemelen gözlemlemeniz gereken şey, yalnızca bir katmanla olduğundan daha yüksek doğruluk elde edebildiğimizdir ve dönem sayısı açısından çok daha hızlıdır - sadece 1 veya 2 dönem. Bu, gelişmiş ağ mimarisinin, neler olup bittiğini anlamak ve imgelerimizden genel örüntüleri çıkarmak için çok daha az veriye ihtiyaç duyduğu anlamına gelir. Ancak, eğitim de daha uzun sürer ve bir GPU gerektirir.

## CIFAR-10 veri kümesinden gerçek imgelerle oynama

El yazısı rakamları tanıma problemimiz bir basit problem gibi görünse de, artık daha ciddi bir şey yapmaya hazırız. Farklı nesnelerin resimlerinden oluşan [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) adlı daha gelişmiş veri kümesini keşfedelim. 10 sınıfa ayrılmış 60 bin 32x32 imge içerir.

In [ ]:
(x_train,y_train),(x_test,y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
display_dataset(x_train,y_train,classes=classes)

CIFAR-10 için iyi bilinen bir mimariye [LeNet](https://en.wikipedia.org/wiki/LeNet) adı verilir ve *Yann LeCun* tarafından önerilmiştir. Yukarıda özetlediğimizle aynı ilkeleri takip eder, ana fark 1 yerine 3 renkli girdi kanalıdır.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters = 6, kernel_size = 5, strides = 1, activation = 'relu', input_shape = (32,32,3)),
    keras.layers.MaxPooling2D(pool_size = 2, strides = 2),
    keras.layers.Conv2D(filters = 16, kernel_size = 5, strides = 1, activation = 'relu'),
    keras.layers.MaxPooling2D(pool_size = 2, strides = 2),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation = 'relu'),
    keras.layers.Dense(84, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')])

model.summary()

Bu ağın düzgün bir şekilde eğitilmesi önemli miktarda zaman alacaktır ve tercihen GPU etkin hesaplama ile yapılmalıdır.

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['acc'])
hist = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10)

In [ ]:
plot_results(hist)

Birkaç eğitim dönemiyle elde edebildiğimiz doğruluk çok büyük görünmüyor. Bununla birlikte, kör tahmininin bize yalnızca %10 doğruluk sağlayacağını ve sorunumuzun aslında MNIST rakam sınıflandırmasından önemli ölçüde daha zor olduğunu unutmayın. Bu kadar kısa bir eğitim süresinde %50'nin üzerine çıkmak iyi bir başarı gibi görünüyor.

## Ana Fikirler

Bu ünitede, bilgisayarla görme sinir ağlarının arkasındaki ana kavramı öğrendik - evrişimli ağlar. İmge sınıflandırmasına, nesne algılamaya ve hatta imge üretme ağlarına güç veren gerçek yaşam mimarilerinin tümü, yalnızca daha fazla katman ve bazı ek eğitim püf noktaları ile CNN'lere dayanmaktadır.